In [156]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import mglearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

# Data Preparation and Feature Engineering:

In [157]:
data = pd.read_csv("Energi_Viborg_Dandas_data.csv")

In [158]:
#drop columns not needed after asking the company about the meaning of these features

columns_to_be_removed = ['ID', 'mslink', 'XKoordinat','YKoordinat','LedningID','Dobbeltled','EjerKompon','SystemKode','KategoriAf','DatoUdf']
data=data.drop(columns_to_be_removed,axis='columns')

In [159]:
# in the column DatoSaneri is the date of repairing and if there is no date it means it is not repaired

data['DatoSaneri'].fillna(0, inplace=True)

In [160]:
# take only the pipes that are broken(by TV insection) now and the repaired ones

data_with_TVObsAndSaneri = data[data['TVObsKode'].isin([1]) | data['DatoSaneri'] > 0]

In [161]:
#get around 2077 rows with not broken pipes

data_not_broken = data[~data['TVObsKode'].isin([0]) | data['DatoSaneri'] == 0]
data_not_broken = data_not_broken.sample(n=2077) 

In [162]:
frames = [data_with_TVObsAndSaneri, data_not_broken]
  
data_final = pd.concat(frames)
data_final

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
36,34.720000,33.48,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,2010,2014,1997.0
42,39.460000,39.16,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
43,39.710000,39.48,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
64,40.550000,40.08,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
65,40.380000,40.55,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22766,53.970000,50.47,112.80,31.028369,200.0,4.0,2017.0,1,0,0.0,2017,2017,0.0
17232,42.585832,42.86,9.58,-28.618802,350.0,1.0,1954.0,1,0,0.0,2012,2014,0.0
5014,14.657335,13.84,33.70,24.253264,250.0,1.0,1941.0,1,10,0.0,2010,2014,0.0
9708,46.390000,43.97,59.59,40.610841,200.0,1.0,1984.0,1,0,0.0,2010,2014,0.0


In [163]:
data = data_final

In [164]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [165]:
#get data copied
datacopy = data


# add  age column

#get current year
from datetime import date
now = date.today().year


def age_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] > 0) :
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])

datacopy['Age'] = datacopy.apply(age_df, axis = 1)

In [166]:
# add a column 'PipeStatus'
# 1 as broken and 0 as not broken

def broken_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] < (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 1
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] >= (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 0
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return 1
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return 0
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return 0

datacopy['PipeStatus'] = datacopy.apply(broken_df, axis = 1)

In [167]:
# datacopy = datacopy.sample(n=22) 
# datacopy

In [168]:
# data_fs= np.where(np.isnan(datacopy))
# data_fs
# row = datacopy.iloc[369] #index=1 => second row
# print(row)

In [169]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(datacopy.shape[0]))
datacopy = datacopy.dropna()
print("Number of rows after removing NaNs: {}".format(datacopy.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [170]:
#drop columns not needed after adding new features

columns_to_be_removed = ['DatoOprett', 'DatoOpdate']
datacopy=datacopy.drop(columns_to_be_removed,axis='columns')
datacopy[0:-1]

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoSaneri,Age,PipeStatus
36,34.720000,33.48,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,1997.0,24.0,0
42,39.460000,39.16,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
43,39.710000,39.48,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
64,40.550000,40.08,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
65,40.380000,40.55,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5774,40.450000,40.20,48.25,5.181347,200.0,1.0,1968.0,1,0,0.0,0.0,53.0,0
22766,53.970000,50.47,112.80,31.028369,200.0,4.0,2017.0,1,0,0.0,0.0,4.0,0
17232,42.585832,42.86,9.58,-28.618802,350.0,1.0,1954.0,1,0,0.0,0.0,67.0,0
5014,14.657335,13.84,33.70,24.253264,250.0,1.0,1941.0,1,10,0.0,0.0,80.0,0


In [171]:
# creating features set and target

columns_to_be_removed = ['Age']
data_features= datacopy.drop(columns_to_be_removed,axis='columns')
columns_to_be_removed = ['fra_kote','til_kote', 'Laengde','Fald','DiameterIn','MaterialeK','anlag_aar','TransportK',
                         'Funktionsk','TVObsKode','DatoSaneri','PipeStatus']
data_target=datacopy.drop(columns_to_be_removed,axis='columns')

In [172]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


# Tuning Alpha for Ridge Model with Train-Test split and Normalization:

In [173]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(data_features, data_target, random_state=42)

In [174]:
alphas = 10**np.linspace(-10, 10, 100)

 # Learn the model with a certain numnber of alphas
ridgecv = RidgeCV(alphas = alphas,scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)
print("Best alpha value found: {}".format(ridgecv.alpha_))

Best alpha value found: 0.0029836472402833404


In [175]:
# coefficients associated with the chosen alpha
ridgecvcoef_ = list(ridgecv.coef_[0])
pd.Series(ridgecvcoef_ , index = data_features.columns)

fra_kote      -0.031802
til_kote       0.041262
Laengde        0.009434
Fald          -0.001925
DiameterIn    -0.001917
MaterialeK     0.029729
anlag_aar     -0.733248
TransportK     0.000000
Funktionsk     0.083590
TVObsKode     -9.780700
DatoSaneri    -0.020509
PipeStatus    13.332544
dtype: float64

In [176]:
# to chech model performance
ridge = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge.fit(X_train, y_train)
mse = mean_squared_error(y_test, ridge.predict(X_test))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.81109946812451


The $R^2$ corresponding to this value for alpha are:

In [177]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(ridge.score(X_train, y_train), 
                                                              ridge.score(X_test,y_test)))

R^2 on train data is 0.8311104945001093 and on test data is 0.8309885299916433


# Tuning Alpha for Ridge Model with validation set split and Normalization:

In [178]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)

In [179]:
best_score = 100
for alphas in 10**np.linspace(-10, 10, 100):
    # Learn the model with a certain numnber of alphas
    ridge1 = Ridge(normalize=True, alpha=alphas)
    ridge1.fit(X_train, y_train)
    
    # Evaluate the model
    score = mean_squared_error(y_val, ridge1.predict(X_val))
    
    
    # If improvement, store score and parameter
    if score < best_score:
        best_score = score
        best_alphas = alphas

# Build a model on the combine training and valiation data
ridge1 = Ridge(normalize=True, alpha = best_alphas)
ridge1.fit(X_trainval, y_trainval)

print("Best alpha found: {}".format(best_alphas))
print("Best MSE on validation set: {}".format(best_score))
print("MSE on training/validation set: {}".format(mean_squared_error(y_trainval, ridge1.predict(X_trainval))))
print("MSE on test set: {}".format(mean_squared_error(y_test, ridge1.predict(X_test))))

Best alpha found: 1e-10
Best MSE on validation set: 90.20294173286142
MSE on training/validation set: 90.45293210769432
MSE on test set: 91.032849028116


In [180]:
# to chech model performance
ridge = Ridge(alpha = best_alphas, normalize = True)
ridge.fit(X_train, y_train)
mse = mean_squared_error(y_test, ridge.predict(X_test))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.07184896703374


In [181]:
# coefficients associated with the chosen alpha
ridge1coef_ = list(ridge.coef_[0])
pd.Series(ridge1coef_ , index = data_features.columns)

fra_kote       0.007794
til_kote       0.000412
Laengde        0.005767
Fald          -0.005658
DiameterIn    -0.002578
MaterialeK     0.066843
anlag_aar     -0.733219
TransportK     0.000000
Funktionsk     0.149071
TVObsKode     -9.490808
DatoSaneri    -0.020660
PipeStatus    13.084698
dtype: float64

In [182]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(ridge.score(X_train, y_train), 
                                                              ridge.score(X_test,y_test)))

R^2 on train data is 0.8317274925326696 and on test data is 0.8323643729736745


# Tuning Alpha for Ridge Model with cross validation split and Normalization:

In [183]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)

In [184]:
best_score = 0
for alphas in 10**np.linspace(-10, 10, 100):
    # Set a certain number of alphas
    ridge1 = Ridge(normalize=True, alpha=alphas)
    
    # Perform cross validation
    scores = cross_val_score(ridge1, X_trainval, y_trainval, cv=5)
    
    # Compute the mean score
    score = scores.mean()
    
    # If improvement, store score and parameter
    if score > best_score:
        best_score = score
        best_alphas = alphas

# Build a model on the combine training and valiation data
ridge1 = Ridge(normalize=True, alpha = best_alphas)
ridge1.fit(X_trainval, y_trainval)

print("Best alpha found: {}".format(best_alphas))
print("Best MSE on validation set: {}".format(best_score))
print("MSE on training/validation set: {}".format(mean_squared_error(y_trainval, ridge1.predict(X_trainval))))
print("MSE on test set: {}".format(mean_squared_error(y_test, ridge1.predict(X_test))))

Best alpha found: 0.001873817422860383
Best MSE on validation set: 0.8281934660273226
MSE on training/validation set: 90.46194072652236
MSE on test set: 90.86201245276217


In [185]:
# to chech model performance
ridge = Ridge(alpha = best_alphas, normalize = True)
ridge.fit(X_train, y_train)
mse = mean_squared_error(y_test, ridge.predict(X_test))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.01881888041491


In [186]:
# coefficients associated with the chosen alpha
ridge1coef_ = list(ridge.coef_[0])
pd.Series(ridge1coef_ , index = data_features.columns)

fra_kote       0.006675
til_kote       0.001198
Laengde        0.005575
Fald          -0.005694
DiameterIn    -0.002555
MaterialeK     0.064600
anlag_aar     -0.730847
TransportK     0.000000
Funktionsk     0.147610
TVObsKode     -8.734977
DatoSaneri    -0.020612
PipeStatus    12.361030
dtype: float64

In [187]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(ridge.score(X_train, y_train), 
                                                              ridge.score(X_test,y_test)))

R^2 on train data is 0.8317126429455503 and on test data is 0.832463068980513


# Tuning Alpha for Ridge Model with Train-Test split and Standardization:

In [188]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(data_features, data_target, random_state=42)

# preprocessing using 0-1 scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [189]:
alphas = 10**np.linspace(-10, 10, 100)

 # Learn the model with a certain numnber of alphas
ridgecv = RidgeCV(alphas = alphas,scoring = 'neg_mean_squared_error')
ridgecv.fit(X_train_scaled, y_train)
print("Best alpha value found: {}".format(ridgecv.alpha_))

Best alpha value found: 8.111308307896856


In [190]:
# ridge.fit(data_features, data_target)
ridgecvcoef_ = list(ridgecv.coef_[0])
pd.Series(ridgecvcoef_ , index = data_features.columns)

fra_kote      -0.525289
til_kote       0.670481
Laengde        0.274561
Fald          -0.122947
DiameterIn    -0.399411
MaterialeK     0.164067
anlag_aar    -14.947931
TransportK     0.000000
Funktionsk     0.126777
TVObsKode     -4.163546
DatoSaneri   -18.777027
PipeStatus     5.551277
dtype: float64

In [191]:
# to chech model performance
ridge = Ridge(alpha = ridgecv.alpha_)
ridge.fit(X_train_scaled, y_train)
mse = mean_squared_error(y_test, ridge.predict(X_test_scaled))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.82620493926125


In [192]:
print("R^2 on train data is {} and on test data is {}".format(ridge.score(X_train_scaled, y_train), 
                                                              ridge.score(X_test_scaled,y_test)))

R^2 on train data is 0.831119159808779 and on test data is 0.8309604167114721


# Tuning Alpha for Ridge Model with validation set split and Standardization:

In [193]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)

# preprocessing using zero mean and unit variance scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
X_trainval_scaled = scaler.transform( X_trainval)


In [194]:
best_score = 100
for alphas in 10**np.linspace(-10, 10, 100):
    # Learn the model with a certain numnber of alphas
    ridge1 = Ridge( alpha=alphas)
    ridge1.fit(X_train_scaled, y_train)
    
    # Evaluate the model
    score = mean_squared_error(y_val, ridge1.predict(X_val_scaled))
    
    
    # If improvement, store score and parameter
    if score < best_score:
        best_score = score
        best_alphas = alphas

# Build a model on the combine training and valiation data
ridge1 = Ridge(alpha = best_alphas)
ridge1.fit(X_trainval_scaled, y_trainval)

print("Best alpha found: {}".format(best_alphas))
print("Best MSE on validation set: {}".format(best_score))
print("MSE on training/validation set: {}".format(mean_squared_error(y_trainval, ridge1.predict(X_trainval_scaled))))
print("MSE on test set: {}".format(mean_squared_error(y_test, ridge1.predict(X_test_scaled))))

Best alpha found: 1e-10
Best MSE on validation set: 90.20294172808507
MSE on training/validation set: 90.45293210769435
MSE on test set: 91.03284904318174


In [195]:
# to chech model performance
ridge = Ridge(alpha = best_alphas)
ridge.fit(X_train_scaled, y_train)
mse = mean_squared_error(y_test, ridge.predict(X_test_scaled))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.0718489708226


In [196]:
# coefficients associated with the chosen alpha
ridge1coef_ = list(ridge.coef_[0])
pd.Series(ridge1coef_ , index = data_features.columns)

fra_kote       0.118811
til_kote       0.006262
Laengde        0.160950
Fald          -0.299858
DiameterIn    -0.529410
MaterialeK     0.349170
anlag_aar    -14.907892
TransportK     0.000000
Funktionsk     0.226772
TVObsKode     -4.003697
DatoSaneri   -18.888154
PipeStatus     5.443152
dtype: float64

In [197]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(ridge.score(X_train_scaled, y_train), 
                                                              ridge.score(X_test_scaled,y_test)))

R^2 on train data is 0.8317274925326696 and on test data is 0.832364372966623


# Tuning Alpha for Ridge Model with cross validation split and Standardization:

In [198]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)

# preprocessing using 0-1 scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_test_scaled = scaler.transform(X_test)
X_trainval_scaled = scaler.transform( X_trainval)

In [199]:
best_score = 0
for alphas in 10**np.linspace(-10, 10, 100):
    # Set a certain number of alphas
    ridge1 = Ridge( alpha=alphas)
    
    # Perform cross validation
    scores = cross_val_score(ridge1, X_trainval_scaled, y_trainval, cv=5)
    
    # Compute the mean score
    score = scores.mean()
    
    # If improvement, store score and parameter
    if score > best_score:
        best_score = score
        best_alphas = alphas

# Build a model on the combine training and valiation data
ridge1 = Ridge(alpha = best_alphas)
ridge1.fit(X_trainval_scaled, y_trainval)

print("Best alpha found: {}".format(best_alphas))
print("Best MSE on validation set: {}".format(best_score))
print("MSE on training/validation set: {}".format(mean_squared_error(y_trainval, ridge1.predict(X_trainval_scaled))))
print("MSE on test set: {}".format(mean_squared_error(y_test, ridge1.predict(X_test_scaled))))

Best alpha found: 5.0941380148163855
Best MSE on validation set: 0.8281950231278632
MSE on training/validation set: 90.45999116870405
MSE on test set: 90.87556801649636


In [200]:
# to chech model performance
ridge = Ridge(alpha = best_alphas)
ridge.fit(X_trainval_scaled,y_trainval)
mse = mean_squared_error(y_test, ridge.predict(X_test_scaled))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.87556801649636


In [201]:
# coefficients associated with the chosen alpha
ridge1coef_ = list(ridge.coef_[0])
pd.Series(ridge1coef_ , index = data_features.columns)

fra_kote      -0.649360
til_kote       0.795608
Laengde        0.271483
Fald          -0.089904
DiameterIn    -0.398850
MaterialeK     0.162579
anlag_aar    -14.944761
TransportK     0.000000
Funktionsk     0.128824
TVObsKode     -4.307857
DatoSaneri   -18.782510
PipeStatus     5.715191
dtype: float64

In [202]:
# R^2 of the associated alpha
print("R^2 on train/validation data is {} and on test data is {}".format(ridge.score(X_trainval_scaled,y_trainval), 
                                                              ridge.score(X_test_scaled,y_test)))

R^2 on train/validation data is 0.8311367549141078 and on test data is 0.8308685454942255
